In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
from src.scripts.pipeline import get_rfm_table, get_most_watch, get_gold_table, get_pivot_table
from src.scripts.support import ingest_from_local
from src.scripts.schema import PL_SCHEMA, Output
from src.helpers.utils import create_date_directories
import polars as pl
import patito as pt

In [4]:
base_path = "../data/log_content/"
start_date = "20220401"
end_date = "20220405"
paths = create_date_directories(base_path, start_date, end_date)
print(paths)

['../data/log_content/20220401.json', '../data/log_content/20220402.json', '../data/log_content/20220403.json', '../data/log_content/20220404.json', '../data/log_content/20220405.json']


In [5]:
sources = ingest_from_local(paths=paths, schema=PL_SCHEMA)

In [22]:
if __name__ == '__main__':
    app_names = [
        "CHANNEL",
        "KPLUS",
        "VOD",
        "FIMS",
        "BHD",
        "SPORT",
        "CHILD",
        "RELAX",
    ]

    column_names = [
        "TVDuration",
        "TVDuration",
        "MovieDuration",
        "MovieDuration",
        "MovieDuration",
        "SportDuration",
        "ChildDuration",
        "RelaxDuration",
    ]
    # get_rfm_table(sources=sources).fetch(100)
    data = get_gold_table(sources, app_names=app_names, columns_names=column_names)
    try:
        Output.validate(data.fetch(100))
    except pt.DataFrameValidationError as e:
        print(e)
        

In [40]:
print(sources.with_columns(
    pl.lit("20220501").alias("reported_date")
).fetch(100))

shape: (500, 10)
┌────────────┬─────────┬─────────┬────────────┬───┬────────────┬────────────┬─────────┬────────────┐
│ Date       ┆ Index   ┆ Type    ┆ Id         ┆ … ┆ Mac        ┆ TotalDurat ┆ AppName ┆ reported_d │
│ ---        ┆ ---     ┆ ---     ┆ ---        ┆   ┆ ---        ┆ ion        ┆ ---     ┆ ate        │
│ datetime[μ ┆ str     ┆ str     ┆ str        ┆   ┆ str        ┆ ---        ┆ str     ┆ ---        │
│ s]         ┆         ┆         ┆            ┆   ┆            ┆ i64        ┆         ┆ str        │
╞════════════╪═════════╪═════════╪════════════╪═══╪════════════╪════════════╪═════════╪════════════╡
│ 2022-04-01 ┆ history ┆ kplus   ┆ AX_momhia1 ┆ … ┆ 0C96E62FC5 ┆ 254        ┆ KPLUS   ┆ 20220501   │
│ 00:00:00   ┆         ┆         ┆ FFivsGrn9o ┆   ┆ 5C         ┆            ┆         ┆            │
│ 2022-04-01 ┆ history ┆ kplus   ┆ AX_momhca1 ┆ … ┆ CCEDDC3336 ┆ 1457       ┆ KPLUS   ┆ 20220501   │
│ 00:00:00   ┆         ┆         ┆ FFivsGrnvg ┆   ┆ 14         ┆          

In [37]:
s = pl.Series(["20220401", "20220401"]).str.strptime(pl.Datetime, format="%Y %m %d")
print(s)

shape: (2,)
Series: '' [datetime[μs]]
[
	2022-04-01 00:00:00
	2022-04-01 00:00:00
]
